In [ ]:
import pandas as pd
from google.colab import drive
import spacy
import numpy as np

In [ ]:
# mount drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_reviews = pd.read_csv("drive/My Drive/Colab Notebooks/TCC/dataset_completed.csv")
df_attractions = pd.read_csv("drive/My Drive/Colab Notebooks/TCC/attractions_list.csv")

In [ ]:
df_reviews_florida = df_reviews[df_reviews["branch"] == "Universal Studios Florida"]
df_reviews_singapore = df_reviews[df_reviews["branch"] == "Universal Studios Singapore"]
df_reviews_japan = df_reviews[df_reviews["branch"] == "Universal Studios Japan"]

df_attractions_florida = df_attractions[df_attractions["branch"] == "Universal Studios Florida"]
df_attractions_singapore = df_attractions[df_attractions["branch"] == "Universal Studios Singapore"]
df_attractions_japan = df_attractions[df_attractions["branch"] == "Universal Studios Japan"]

In [ ]:
lemma = spacy.load('en_core_web_sm')

stop_words = ["attraction","park", "time", "guest", "ride", "studios", "coaster", 
              "food", "universal", "orlando", "express", "adventure", "trip", 
              "amazing", "studio", "fun", "shop", "street", "bar", "water", 
              "movie", "space", "big", "little", "night", "day", "place", "street"]

In [ ]:
#using nparray to decrease execution time
def get_attractions(df_reviews, df_attractions):
  attractions = []
  for index in np.nditer(df_reviews["tokens"].values, order="C", flags=["refs_ok"]):
    tokens = index.tolist()
    tokens_attractions = []
    for attraction in np.nditer(df_attractions["name"].values, order="C", flags=["refs_ok"]):
      if attraction not in tokens_attractions:
        splitted = str(attraction).split()
        for word in splitted:
          if not lemma.vocab[word].is_stop and lemma.vocab[word].is_alpha and word not in stop_words and str(word) in tokens:
            #print(f'attraction found {attraction} with word {word} in token {tokens}')
            tokens_attractions.append(attraction)
            #print(f'size of tokens attractions {len(tokens_attractions)}')
    attractions.append(tokens_attractions)
  df_reviews["attractions"] = attractions
  return df_reviews

In [ ]:
df_florida = get_attractions(df_reviews_florida, df_attractions_florida)
df_singapore = get_attractions(df_reviews_singapore, df_attractions_singapore)
df_japan = get_attractions(df_reviews_japan, df_attractions_japan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [ ]:
df_final_with_attractions = pd.concat([df_florida, df_singapore, df_japan])
df_final_with_attractions = df_final_with_attractions.reset_index()

In [ ]:
# Gerando dataset final
df_final_with_attractions.to_csv("drive/My Drive/Colab Notebooks/TCC/dataset_completed_attractions.csv",index=False)